### Bandit Problem


상호 관계에 바탕을 둔 강화학습의 프레임워크

- Agent : 행위자
- Action : 행위
- Environment : World
- State : 환경의 상태
- Reward : 보상


![Image of Yaktocat](https://mblogthumb-phinf.pstatic.net/MjAxNzEwMTZfMTc2/MDAxNTA4MTE5MzMzOTgx.5SwCUZzPqMUqDw3y_NuGFZbhNv7Yaby3DPn9GJLpckkg.jHIdTSJjmjvZnOp9FkGyyj-zdhTkKht6Okpi-2FJCzgg.PNG.linegamedev/1.png?type=w800)



### 강화학습의 세가지 요소
- 액션 의존성 : 각 액션은 다른 보상을 가져오게 됩니다.
- 시간 의존성 : 보상은 시간이 지연되고 나서야 주어집니다.
- 상태 의존성 : 어떤 액션에 대한 보상은 환경 상태에 좌우됩니다.

우리가 최적의 Action을 선택하도록 보장하는 것, 이를 $정책^{Policy}$이라고 한다.

### Policy Gradient

비용을 수식으로 다음과 같이 표현

$$
Loss = - \log(\pi)* A
$$

$\pi$ : Policy  
$A$ : Advantage


<img src="https://github.com/DeepHaeJoong/Deep-ReinForcement/blob/master/PNG/Bandit%20Problem.PNG?raw=true" alt="drawing" width="600"/>

### 환경은 상태를 가지지 않으며 Agent는 단순히 최선의 Action을 하도록 학습하면 된다.

In [262]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np


# 밴딧의 손잡이 목록을 작성한다.
# 현재 손잡이 4(인덱스 3)가 가장 자주 양의 보상을 제공하도록 설정되어 있다.
bandit_arms = [0.2, 0, -0.2, -1]
num_arms = len(bandit_arms)

def pullBandit(bandit):
    # 랜덤한 값을 구한다.
    result = np.random.randn(1)
    if result > bandit:
        # 양의 보상을 반환 한다.
        return 1
    else:
        # 음의 보상을 반환 한다.
        return -1

### Agent

각 벤딧 손잡이에 대한 일련의 값들로 구성되어 있음. 각 값은 해당 벤딧을 선택할 때 반환되는 보상의 추정값을 의미한다. 우리는 저액 경사 방법을 이용해, 선택된 액션에 대해 큰 보상을 받는 쪽으로 디동해나가며 에이전트를 업데이트.

In [263]:
tf.reset_default_graph()

# 네트워크의 피드포워드 부분을 구현한다.
weights = tf.Variable(tf.ones([num_arms])) # [1. 1. 1. 1.]
output = tf.nn.softmax(weights)            # [0.25 0.25 0.25 0.25]


# 학습 과정을 구현
# 보상과 선택된 액션을 네트워크에 비드해줌으로써 비용을 계산하고
# 비용을 이용해 네트워크를 업데이트 한다.
reward_holder = tf.placeholder(shape = [1], dtype = tf.float32)
action_holder = tf.placeholder(shape = [1], dtype = tf.int32)


responsible_output = tf.slice(output, action_holder, [1]) # (inpit, begin, size) -> 
loss = -(tf.log(responsible_output)*reward_holder)
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3)
update = optimizer.minimize(loss)

In [260]:
print(actions)
a = np.random.choice(actions, p = actions)
print(a)
action = np.argmax(actions == a)
print(action)

[0.20235686 0.22514232 0.23405087 0.33844995]
0.33844995
3


### Agent 학습시키기

In [269]:
# Agent를 학습시킬 총 에피소드의 수를 설정한다.
total_episodes = 1000

# 벤딧 손잡이에 대한 점수판을 0으로 설정
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

# 텐서플로 그래프 론칭
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        # 볼츠만 분포에 따라 액션 선택
        actions = sess.run(output)
        a = np.random.choice(actions, p = actions)  # 선택 확률을 다르게 해서 1개 선택
        action = np.argmax(actions == a)            # 해당 index 출력
        
        
        # 벤딧 손잡이 중 하나를 선택함으로써 보상을 받는다.
        reward = pullBandit(bandit_arms[action])  # [0.2, 0, -0.2, -1][index]를 통해 난수보다 작으면 1 크면 -1 받음
        
        
        # 네트워크를 업데이트 한다.
        _, resp, ww = sess.run([update, responsible_output, weights], feed_dict = {reward_holder:[reward], action_holder:[action]})

        
        # 보상의 총계 업데이트
        total_reward[action] += reward
        if i%50 == 0:
            print("Running reward for the " + str(num_arms) + " arms of the bandit: " + str(total_reward))
#            print("가중치 변화 : ",ww)
#            print("resp : ", resp)
        i+=1
        
        
print("\n The agent thinks arm " + str(np.argmax(ww)+1) +" is the most promissing...")


if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("...and it was right!")
else:
    print("...and it was wrong!")

Running reward for the 4 arms of the bandit: [-1.  0.  0.  0.]
Running reward for the 4 arms of the bandit: [ 2. -1. -4.  4.]
Running reward for the 4 arms of the bandit: [ 5.  2. -6. 14.]
Running reward for the 4 arms of the bandit: [ 2. -2. -2. 27.]
Running reward for the 4 arms of the bandit: [-5. -3.  2. 37.]
Running reward for the 4 arms of the bandit: [-4.  0.  4. 53.]
Running reward for the 4 arms of the bandit: [-7.  6.  8. 60.]
Running reward for the 4 arms of the bandit: [-5.  5.  9. 66.]
Running reward for the 4 arms of the bandit: [-5.  2.  2. 74.]
Running reward for the 4 arms of the bandit: [-6. -1.  3. 85.]
Running reward for the 4 arms of the bandit: [-9. -6. 11. 91.]
Running reward for the 4 arms of the bandit: [-13.  -7.  15.  96.]
Running reward for the 4 arms of the bandit: [-13.  -1.  20. 107.]
Running reward for the 4 arms of the bandit: [-17.   5.  22. 119.]
Running reward for the 4 arms of the bandit: [-21.   3.  24. 133.]
Running reward for the 4 arms of the ba